# Data Pull

In [1]:
# Import Pandas and Numpy. Set DataFrame Viewing Window. 
import pandas as pd
import numpy as np
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [1]:
# Pull Final Variable List 
sf = spark.sql("""SELECT
expdemo_census_pct_parent_fmly_hh,
expdemo_census_perc_hu_blt_1970_1979,
expdemo_consumer_avg_hh_size,
expdemo_consumer_brokerage_act_owner,
expdemo_consumer_freq_flyer_prog_mem,
expdemo_consumer_intrst_snow_sports_ai,
expdemo_consumer_median_fam_hh_income,
expdemo_consumer_pct_hh_marcplfam, 
expdemo_pscs_pct_wrst_status_3024,
expdemo_pscs_avg_num_credit_inq_6mo, 
target
FROM cxue200.allvars_pros_xm_npdchurn_201903 
""")
df = sf.toPandas() 
df.head() 


,expdemo_census_pct_parent_fmly_hh,expdemo_census_perc_hu_blt_1970_1979,expdemo_consumer_avg_hh_size,expdemo_consumer_brokerage_act_owner,expdemo_consumer_freq_flyer_prog_mem,expdemo_consumer_intrst_snow_sports_ai,expdemo_consumer_median_fam_hh_income,expdemo_consumer_pct_hh_marcplfam,expdemo_pscs_pct_wrst_status_3024,expdemo_pscs_avg_num_credit_inq_6mo,target
0,0.3,6.5,2.16,76.0,75.0,3.0,69592.0,42.6,33.00,0.50,0
1,0.8,9.7,2.27,10.0,28.0,84.0,74062.0,58.2,50.00,0.67,0
2,0.4,31.1,2.78,41.0,73.0,68.0,95075.0,60.6,58.00,0.42,0
3,0.7,24.2,2.51,71.0,69.0,36.0,49323.0,40.8,64.00,0.25,0
4,1.0,5.8,2.84,62.0,38.0,3.0,95518.0,66.6,20.00,0.17,0


In [ ]:
# Pull OOT1
oot1 = spark.sql("""SELECT 
expdemo_census_pct_parent_fmly_hh,
expdemo_census_perc_hu_blt_1970_1979,
expdemo_consumer_avg_hh_size,
expdemo_consumer_brokerage_act_owner,
expdemo_consumer_freq_flyer_prog_mem,
expdemo_consumer_intrst_snow_sports_ai,
expdemo_consumer_median_fam_hh_income,
expdemo_consumer_pct_hh_marcplfam, 
expdemo_pscs_pct_wrst_status_3024,
expdemo_pscs_avg_num_credit_inq_6mo, 
target
FROM mmanle240.final_pro_npd_oot1_labeled""") 
oot1_p = oot1.toPandas() 
oot1_p.head() 

In [ ]:
# Pull OOT2
oot2 = spark.sql("""SELECT 
expdemo_census_pct_parent_fmly_hh,
expdemo_census_perc_hu_blt_1970_1979,
expdemo_consumer_avg_hh_size,
expdemo_consumer_brokerage_act_owner,
expdemo_consumer_freq_flyer_prog_mem,
expdemo_consumer_intrst_snow_sports_ai,
expdemo_consumer_median_fam_hh_income,
expdemo_consumer_pct_hh_marcplfam, 
expdemo_pscs_pct_wrst_status_3024,
expdemo_pscs_avg_num_credit_inq_6mo, 
target
FROM mmanle240.pro_xm_npd_churn_oot3""") 
oot2_p = oot2.toPandas() 
oot2_p.head() 

In [5]:
# Encode Categorical 
from sklearn import preprocessing
from collections import defaultdict

df = df.fillna(0)
def code(frame):
    d = defaultdict(preprocessing.LabelEncoder)
    fit = frame.select_dtypes(exclude=['int']).fillna('NA').apply(lambda x: d[x.name].fit_transform(x))
    for i in list(d.keys()): 
        frame[i] = d[i].transform(frame[i].fillna('NA'))
    return frame


encs = [
'expdemo_census_pct_parent_fmly_hh',
'expdemo_census_perc_hu_blt_1970_1979', 
'expdemo_consumer_avg_hh_size',
'expdemo_consumer_pct_hh_marcplfam',
'expdemo_pscs_pct_wrst_status_3024',
'expdemo_pscs_avg_num_credit_inq_6mo'] 


df[encs] = code(df[encs])



/opt/cloudera/parcels/Anaconda-4.3.1/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
# Seperate DataFrame into features and target
features = df[df.columns.difference(['target'])]
labels = df['target']
features_list = features.columns

In [ ]:
# Encode and replace missing values OOT
oot1_p = oot1_p.fillna(0)
oot1_p[encs] = code(oot1_p[encs])
oot2_p = oot2_p.fillna(0)
oot2_p[encs] = code(oot2_p[encs])


label_test_oot1 = oot1_p['target'] 
features_test_oot1 = oot1_p[feature_list]
label_test_oot2 = oot2_p['target'] 
features_test_oot2 = oot2_p[feature_list]

# Variable Selection

In [52]:
# Use to Calculate WOE
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [219]:
# WOE Full Results
final_iv, IV = data_vars(df[df.columns.difference(['target'])],df.target)
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,expdemo_census_pct_parent_fmly_hh,0.00,12.00,179715,1036.00,0.01,178679.00,0.99,0.53,0.51,0.03,0.00
1,expdemo_census_pct_parent_fmly_hh,13.00,83.00,169606,916.00,0.01,168690.00,0.99,0.47,0.49,-0.03,0.00
2,expdemo_census_perc_hu_blt_1970_1979,0.00,116.00,175905,1007.00,0.01,174898.00,0.99,0.52,0.50,0.02,0.00
3,expdemo_census_perc_hu_blt_1970_1979,117.00,895.00,173416,945.00,0.01,172471.00,0.99,0.48,0.50,-0.03,0.00
4,expdemo_consumer_avg_hh_size,0.00,147.00,176398,947.00,0.01,175451.00,0.99,0.49,0.51,-0.04,0.00
5,expdemo_consumer_avg_hh_size,148.00,414.00,172923,1005.00,0.01,171918.00,0.99,0.51,0.49,0.04,0.00
6,expdemo_consumer_brokerage_act_owner,0.00,26.00,118330,327.00,0.00,118003.00,1.00,0.17,0.34,-0.71,0.37
7,expdemo_consumer_brokerage_act_owner,27.00,62.00,117068,417.00,0.00,116651.00,1.00,0.21,0.34,-0.45,0.37
8,expdemo_consumer_brokerage_act_owner,63.00,99.00,113923,1208.00,0.01,112715.00,0.99,0.62,0.32,0.65,0.37
9,expdemo_consumer_census_sub_tract_2010,0.00,1.00,181461,1140.00,0.01,180321.00,0.99,0.58,0.52,0.12,0.02


In [220]:
# WOE IV
IV = IV.rename(columns={'VAR_NAME':'index'})
IV.sort_values(['IV'],ascending=0)

,index,IV
17,expdemo_pscs_pct_wrst_crdt_30dpd,0.79
18,expdemo_pscs_pct_wrst_status_30,0.74
19,expdemo_pscs_pct_wrst_status_3024,0.72
16,expdemo_pscs_pct_trades_delinq30,0.69
14,expdemo_pscs_low_credit_seg,0.64
11,expdemo_pscs_avg_bcc_trd,0.61
15,expdemo_pscs_num_rtrade,0.55
7,expdemo_consumer_median_fam_hh_income,0.50
12,expdemo_pscs_avg_credit_6mo,0.47
8,expdemo_consumer_pct_hh_marcplfam,0.42


In [105]:
# Train RF 
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

clf.fit(features,labels)

preds = clf.predict(features)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(preds,labels)
print(accuracy)

0.999029545890456


In [106]:
# Display RF VI
from pandas import DataFrame
VI = DataFrame(clf.feature_importances_, columns = ["RF"], index=features.columns)
VI = VI.reset_index()
VI.sort_values(['RF'],ascending=0)

In [108]:
# Train ExtraTrees
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(features, labels)

print(model.feature_importances_)

[0.01646826 0.01374572 0.01420418 0.01445457 0.01332583 0.01452174
 0.01443888 0.01392105 0.01485994 0.01503735 0.01471621 0.01445089
 0.01472528 0.01547152 0.01357211 0.01372832 0.01439683 0.01429821
 0.01408299 0.01393751 0.01373761 0.01418969 0.01322968 0.01281567
 0.0128185  0.01537464 0.01405158 0.01555498 0.01072561 0.01262355
 0.0116166  0.01255857 0.0101316  0.01126134 0.01170587 0.01131395
 0.01090745 0.0106196  0.01048934 0.01329594 0.01211746 0.01081543
 0.01275979 0.00875794 0.0143093  0.01279625 0.01129996 0.01224956
 0.01070569 0.01185852 0.01140832 0.01273451 0.01151698 0.01163376
 0.01165304 0.01249992 0.0120477  0.01243282 0.01257856 0.01287645
 0.01181216 0.01149741 0.01232051 0.01226072 0.01174966 0.011597
 0.01339961 0.01154385 0.01173071 0.01542989 0.02437505 0.01598008
 0.01472561 0.01596549 0.01447571 0.01470545]


In [109]:
# Display ExtraTrees VI
from pandas import DataFrame
FI = DataFrame(model.feature_importances_, columns = ["Extratrees"], index=features.columns)
FI = FI.reset_index()
FI.sort_values(['Extratrees'],ascending=0)

In [112]:
# Chi Square Test
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

model = SelectKBest(score_func=chi2, k=5)
fit = model.fit(features.abs(), labels)

from pandas import DataFrame
pd.options.display.float_format = '{:.2f}'.format
chi_sq = DataFrame(fit.scores_, columns = ["Chi_Square"], index=features.columns)

chi_sq = chi_sq.reset_index()
chi_sq.sort_values('Chi_Square',ascending=0)

In [115]:
# Linear SVC Reduction 
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(features, labels)
model = SelectFromModel(lsvc,prefit=True)

from pandas import DataFrame
l1 = DataFrame(model.get_support(), columns = ["L1"], index=features.columns)
l1 = l1.reset_index()

l1[l1['L1'] == True]

In [118]:
# Create Results Table from all Selection Methods 
from functools import reduce
dfs = [IV, VI, FI, chi_sq, l1]
final_results = reduce(lambda left,right: pd.merge(left,right,on='index'), dfs)

columns = ['IV', 'RF', 'Extratrees', 'Chi_Square']

score_table = pd.DataFrame({},[])
score_table['index'] = final_results['index']

for i in columns:
    score_table[i] = final_results['index'].isin(list(final_results.nlargest(10,i)['index'])).astype(int)
    
score_table['L1'] = final_results['L1'].astype(int)

score_table['final_score'] = score_table.sum(axis=1)
score_table.sort_values('final_score',ascending=0)

In [133]:
# Reduce based on results
features = features[list(score_table[score_table['final_score'] >= 1]['index'])]
features.head()

,expdemo_census_pct_commute_wrk_lt_30,expdemo_census_pct_hh_income_lt_10k,expdemo_census_pct_parent_fmly_hh,expdemo_census_pct_pop_two_plus_races,expdemo_census_perc_hu_blt_1970_1979,expdemo_census_personshh_pophh,expdemo_consumer_avg_hh_size,expdemo_consumer_brokerage_act_owner,expdemo_consumer_census_sub_tract_2010,expdemo_consumer_credit_crd_user,expdemo_consumer_freq_flyer_prog_mem,expdemo_consumer_high_freq_cruise_enth,expdemo_consumer_intrsn_plays_tenni_ai,expdemo_consumer_intrst_ebook_read_ai,expdemo_consumer_intrst_gourment_cook_ai,expdemo_consumer_intrst_healthy_liv_ai,expdemo_consumer_intrst_snow_sports_ai,expdemo_consumer_median_age_hu,expdemo_consumer_median_educ_pop_25plus,expdemo_consumer_median_fam_hh_income,expdemo_consumer_mosaic_2011_intl,expdemo_consumer_music_lis_country_ai,expdemo_consumer_music_lis_rock_ai,expdemo_consumer_pct_hh_marcplfam,expdemo_consumer_predict_amt_donation,expdemo_pscs_avg_bal_excl_coll,expdemo_pscs_avg_bcc_creditbal100_6mo,expdemo_pscs_avg_bcc_never_delinq,expdemo_pscs_avg_bcc_trd,expdemo_pscs_avg_credit_6mo,expdemo_pscs_avg_credit_amt_recent,expdemo_pscs_avg_credit_amt_report_6,expdemo_pscs_avg_credit_amt_revolve_6,expdemo_pscs_avg_num_credit_inq_6mo,expdemo_pscs_bal_credit_rtrade_6mo,expdemo_pscs_credit_amt_rtrade_6mo,expdemo_pscs_low_credit_seg,expdemo_pscs_num_credit_6mo,expdemo_pscs_num_rtrade,expdemo_pscs_num_trade_delinq30,expdemo_pscs_num_trade_delinq90_6mo,expdemo_pscs_num_trade_extdelinq,expdemo_pscs_num_tradepr_delinq90,expdemo_pscs_pct_iln_current_36mo,expdemo_pscs_pct_trades_delinq30,expdemo_pscs_pct_trades_delinq60,expdemo_pscs_pct_trades_derog_excl_coll,expdemo_pscs_pct_worst_status_24,expdemo_pscs_pct_worst_status_2430,expdemo_pscs_pct_wrst_crdt,expdemo_pscs_pct_wrst_crdt_30dpd,expdemo_pscs_pct_wrst_status,expdemo_pscs_pct_wrst_status_30,expdemo_pscs_pct_wrst_status_3024,expdemo_pscs_pct_wrst_status_36,sciera_max_internet_speed,tts_past_billing_sol_cnt,videopenetrationrate,wt_avg_node_num_cm_online_l30d,wt_avg_node_num_res_affected_l3d,wt_max_node_nationalnumeventsfound_l7d
128037,324,35,7,8,278,141,190,10.00,2,1.00,1.00,14.00,34.00,7.00,79.00,31.00,21.00,15.00,68,92003.00,1,41.00,24.00,782,1670.00,0,0,259,293,133420,32071,0,2562,50,83266,47933,8.00,82017,466,299,0,56,113,446,440,697,697,83,7,83,33,69,35,26,78,35,0.00,1516,150.80,8.62,6.00
491755,396,50,21,40,113,204,209,52.00,17,31.00,29.00,24.00,15.00,45.00,63.00,69.00,31.00,31.00,65,83467.00,1,66.00,59.00,650,2871.00,6835,23,437,487,127837,47372,96022,82795,53,135596,85940,6.00,123903,910,143,19,85,101,420,2033,1663,1418,65,19,66,34,53,46,31,76,12,0.00,2508,94.04,15.00,1.00
470924,414,44,55,62,180,210,215,10.00,1,14.00,13.00,39.00,12.00,61.00,57.00,38.00,16.00,16.00,62,86810.00,2,64.00,25.00,583,2263.00,0,14,537,579,151912,62616,148711,103426,36,112443,56101,7.00,140989,898,113,79,21,100,446,747,640,269,79,8,93,7,71,28,20,87,16,1.00,755,0.00,0.00,0.00
491263,436,53,11,63,416,110,115,0.00,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,47.00,92,139642.00,0,0.00,0.00,450,0.00,4097,18,435,474,95411,64951,129987,123066,49,97145,55447,6.00,116801,696,108,18,44,58,394,1613,1225,973,83,6,83,17,56,43,13,89,9,1.00,2755,100.51,9.00,1.00
371403,313,12,49,68,7,231,236,11.00,52,2.00,1.00,13.00,2.00,9.00,47.00,50.00,7.00,11.00,87,183413.00,2,63.00,50.00,753,6143.00,0,0,576,627,209681,81502,192193,150597,45,177634,104895,8.00,198831,809,47,0,23,27,446,322,240,153,93,11,95,5,82,17,6,84,13,0.00,3065,38.44,6.00,1.00


#  Modeling

In [10]:
# Split into train and Split 
from sklearn.cross_validation import train_test_split

train, test = train_test_split(df, test_size = 0.2, random_state=1)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

features_train = train[feature_list]
label_train = train['target']
features_test = test[feature_list]
label_test = test['target']

In [6]:
# Load Model if already trained
import pickle
filename=open('ebi_n19_08_007.pkl', 'rb') 
clf = pickle.load(filename) 

/opt/cloudera/parcels/Anaconda/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
# Spark KS Function
import pandas as pd
from pyspark.sql.types import DoubleType
from pyspark.sql.window import *
from pyspark.sql.functions import desc
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql import functions as func

 

# Function to calculate KS chart
def ks_chart(data):
    #model_score = data.select('prob_a1')
    wAllReg =  Window.orderBy(desc('proba_1'))
    decile_data=data.withColumn('NATIONAL_DECILE',ntile(10).over(wAllReg))
    decile_data=decile_data.withColumn('non_target',1-decile_data['target'])

 

    target_cnt=decile_data.groupBy('NATIONAL_DECILE').agg(func.sum('target').alias('target_sum'))
    target_cnt=target_cnt.withColumnRenamed('NATIONAL_DECILE','F1')
    non_target_cnt=decile_data.groupBy('NATIONAL_DECILE').agg(func.sum("non_target").alias('non_target_sum'))
    non_target_cnt=non_target_cnt.withColumnRenamed('NATIONAL_DECILE','F2')
    overall_cnt=decile_data.groupBy('NATIONAL_DECILE').count().alias('Total')
    overall_cnt=overall_cnt.join(target_cnt,target_cnt.F1==overall_cnt.NATIONAL_DECILE)
    overall_cnt=overall_cnt.join(non_target_cnt,non_target_cnt.F2==overall_cnt.NATIONAL_DECILE)
    overall_cnt=overall_cnt.withColumn("Pct_targetR",(func.col('target_sum')/func.col('count'))*100)

 

    #cumulative functions
    overall_cnt=overall_cnt.withColumn('new_column', func.lit(1))
    cumSumPartition = Window.partitionBy(overall_cnt.new_column).orderBy("NATIONAL_DECILE")
    overall_cnt = overall_cnt.withColumn("cum_target",sum(overall_cnt.target_sum).over(cumSumPartition))
    overall_cnt = overall_cnt.withColumn("cum_non_target",sum(overall_cnt.non_target_sum).over(cumSumPartition))
#Distribution
    tt =overall_cnt.agg(func.sum('target_sum').alias('tar_div'))
    tt1 =overall_cnt.agg(func.sum('non_target_sum').alias('non_tar_div'))
    spark.conf.set("spark.sql.crossJoin.enabled", "true")
    overall_cnt= overall_cnt.crossJoin(tt)
    overall_cnt= overall_cnt.crossJoin(tt1)
    spark.conf.set("spark.sql.crossJoin.enabled", "false")
    overall_cnt=overall_cnt.withColumn("Dist_target",(func.col('cum_target')/func.col('tar_div'))*100)
    overall_cnt=overall_cnt.withColumn("Dist_non_target",(func.col('cum_non_target')/func.col('non_tar_div'))*100)
    #spread
    overall_cnt= overall_cnt.withColumn("SpreadR",(overall_cnt['Dist_target']-overall_cnt['Dist_non_target']))
    #roundup
    overall_cnt = overall_cnt.withColumn('%Dist_target', round(overall_cnt.Dist_target, 2))
    overall_cnt = overall_cnt.withColumn('%Dist_non_target', round(overall_cnt.Dist_non_target, 2))
    overall_cnt = overall_cnt.withColumn('Pct_target', round(overall_cnt.Pct_targetR, 2))
    overall_cnt = overall_cnt.withColumn('Spread', round(overall_cnt.SpreadR, 2))
    #KS-Value
    KS_Value= overall_cnt.agg({"Spread": "max"}).collect()[0]
    overall_cnt=overall_cnt.drop('F1').drop('F2').drop('new_column').drop('tar_div').drop('non_tar_div').drop('Dist_target').drop('Dist_non_target').drop("Pct_targetR").drop("SpreadR")
    overall_cnt=overall_cnt.sort('NATIONAL_DECILE')
    pf = overall_cnt.toPandas() 
    ks = "KS_Value =" + str(KS_Value[0]) 
    return pf, str(KS_Value[0])



In [42]:
# Evaluate AUC on Test
from sklearn import metrics
import numpy as np

fpr, tpr, _ = metrics.roc_curve(np.array(label_test), clf.predict_proba(features_test)[:,1])
auc_test = metrics.auc(fpr,tpr)
print(auc_test)

0.7389860855496266


In [18]:
# Evaluate KS on Test 
probas_pos = clf.predict_proba(features_test)[:,1]
test_Y = label_test
predictions = clf.predict(features_test)

# Make scoring dataframe
scored = pd.DataFrame(
    {'proba_1': probas_pos,
     'target': test_Y
    })
scored_df = spark.createDataFrame(scored)


char, ks = ks_chart(scored_df) 
print(ks)

39.14


In [14]:
# AUC OOT1
pred_test_oot1 = clf.predict(features_test_oot1)

from sklearn.metrics import accuracy_score
accuracy_test = accuracy_score(pred_test_oot1,label_test_oot1)



fpr, tpr, _ = metrics.roc_curve(np.array(label_test_oot1), clf.predict_proba(features_test_oot1)[:,1])
auc_test = metrics.auc(fpr,tpr)

print(accuracy_test,auc_test)



(0.9948056481804721, 0.7217727678303204)


In [180]:
# KS OOT1
import pandas as pd
probas_pos = clf.predict_proba(features_test_oot1)[:,1]
predictions = clf.predict(features_test_oot1)
test_Y = label_test_oot1

# Make scoring dataframe
scored = pd.DataFrame(
    {'proba_1': probas_pos,
     'target': test_Y
    })
scored_df = spark.createDataFrame(scored)


char, ks = ks_chart(scored_df)
print(ks)

34.4


In [22]:
# AUC OOT2
import numpy as np
from sklearn import metrics
pred_test_oot2 = clf.predict(features_test_oot2)

from sklearn.metrics import accuracy_score
accuracy_test = accuracy_score(pred_test_oot2,label_test_oot2)



fpr, tpr, _ = metrics.roc_curve(np.array(label_test_oot2), clf.predict_proba(features_test_oot2)[:,1])
auc_test = metrics.auc(fpr,tpr)

print(accuracy_test,auc_test)

(0.9659774928029312, 0.6352145551549506)


In [17]:
# KS OOT2
import pandas as pd
probas_pos = clf.predict_proba(features_test_oot2)[:,1]
predictions = clf.predict(features_test_oot2)
test_Y = label_test_oot2

# Make scoring dataframe
scored = pd.DataFrame(
    {'proba_1': probas_pos,
     'target': test_Y
    })
scored_df = spark.createDataFrame(scored)


char, ks = ks_chart(scored_df)
print(ks)

20.71


# Documentation 

In [ ]:
# Division Decile

for div in div_list: 
    features = df[df['serloc_current_division_name'] == div]
    #labels = features['target']
    #features = features.drop(['target', 'serloc_current_division_name', 'serloc_current_region_name'], axis=1)


    train, test = train_test_split(features, test_size = 0.2, random_state=1)
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)

    
    features_test = test[feature_list]
    label_test = test['target']



    probas_pos = clf.predict_proba(features_test)[:,1]
    test_Y = label_test
    scored = pd.DataFrame(
            {'proba_1': probas_pos,
             'target': test_Y
            })
    scored_df = spark.createDataFrame(scored)

    char, ks = ks_chart(scored_df) 
    print(div)
    print(ks)
    for item in char['count']: 
        print(item) 
    print("\n") 
    for item in char['target_sum']: 
        print(item)

In [ ]:
# Region 

for reg in reg_list: 
    features = df[df['serloc_current_region_name'] == reg]
    #labels = features['target']
    #features = features.drop(['target', 'serloc_current_division_name', 'serloc_current_region_name'], axis=1)


    train, test = train_test_split(features, test_size = 0.2, random_state=1)
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)

    
    features_test = test[feature_list]
    label_test = test['target']



    probas_pos = clf.predict_proba(features_test)[:,1]
    test_Y = label_test
    scored = pd.DataFrame(
            {'proba_1': probas_pos,
             'target': test_Y
            })
    scored_df = spark.createDataFrame(scored)

    char, ks = ks_chart(scored_df) 
    print(reg)
    print(ks)
    for item in char['count']: 
        print(item) 
    print("\n") 
    for item in char['target_sum']: 
        print(item)

In [146]:
# Run Logistic. Print Report.
logit_model=sm.Logit(df['target'], df[feature_list])
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.033962
         Iterations 10
                                    Results: Logit
Model:                      Logit                   Pseudo R-squared:        -0.009    
Dependent Variable:         target                  AIC:                     47474.9129
Date:                       2019-09-05 14:46        BIC:                     47589.4818
No. Observations:           698641                  Log-Likelihood:          -23727.   
Df Model:                   9                       LL-Null:                 -23504.   
Df Residuals:               698631                  LLR p-value:             1.0000    
Converged:                  1.0000                  Scale:                   1.0000    
No. Iterations:             10.0000                                                    
---------------------------------------------------------------------------------------
                                        Coef.  Std.Err

In [148]:
# Chi
import sklearn.feature_selection 
from sklearn.feature_selection import chi2
for c in df[feature_list].columns: 
    chip, chipe = chi2(df[[c]], df['target']) 
    print(c, chip)

('expdemo_census_pct_parent_fmly_hh', array([0.75169918]))
('expdemo_census_perc_hu_blt_1970_1979', array([165.36649498]))
('expdemo_consumer_avg_hh_size', array([232.90420601]))
('expdemo_consumer_brokerage_act_owner', array([33132.38404838]))
('expdemo_consumer_freq_flyer_prog_mem', array([24097.88304012]))
('expdemo_consumer_intrst_snow_sports_ai', array([4591.57863435]))
('expdemo_consumer_median_fam_hh_income', array([22571874.01495433]))
('expdemo_consumer_pct_hh_marcplfam', array([80670.22575589]))
('expdemo_pscs_avg_num_credit_inq_6mo', array([5572.02002354]))
('expdemo_pscs_pct_wrst_status_3024', array([37099.48018803]))


In [150]:
# VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calculate_vif(features):
    vif = pd.DataFrame()
    vif["Features"] = features.columns
    vif["VIF"] = [variance_inflation_factor(features.values, i) for i in range(features.shape[1])]    
    return(vif) 
calculate_vif(df[feature_list])

,Features,VIF
0,expdemo_census_pct_parent_fmly_hh,3.685929
1,expdemo_census_perc_hu_blt_1970_1979,2.132115
2,expdemo_consumer_avg_hh_size,22.082972
3,expdemo_consumer_brokerage_act_owner,10.391748
4,expdemo_consumer_freq_flyer_prog_mem,11.085219
5,expdemo_consumer_intrst_snow_sports_ai,4.454781
6,expdemo_consumer_median_fam_hh_income,8.858628
7,expdemo_consumer_pct_hh_marcplfam,25.493020
8,expdemo_pscs_avg_num_credit_inq_6mo,5.374694
9,expdemo_pscs_pct_wrst_status_3024,4.888754


In [4]:
# Dev targets
df['target'].value_counts()

0    694860
1      3781
Name: target, dtype: int64

In [5]:
# OOT1 targets
oot1_p['target'].value_counts() 

0    661120
1      3448
Name: target, dtype: int64

In [6]:
# OOT2 targets
oot2_p['target'].value_counts()

0    3691
1     130
Name: target, dtype: int64

In [150]:
# Create dataframe with all features and scores
probs = pd.DataFrame({'prob':probas_pos, 'pred': predictions}) 
results_test = features_test_oot1.join(probs, how='left') 
results_test = results_test.join(test_Y, how='left')
results_sf = spark.createDataFrame(results_test)

In [153]:
# Use to find  predicted vs actual per decile.
def percents(data, model_score, target):
    wAllReg =  Window.orderBy(desc('prob'))
    decile_data=data.withColumn('NATIONAL_DECILE',ntile(10).over(wAllReg))
    decile_data=decile_data.withColumn('non_target',1-decile_data['target'])

    target_cnt=decile_data.groupBy('NATIONAL_DECILE').agg(func.sum('target').alias('target_sum_total'))
    target_cnt=target_cnt.withColumnRenamed('NATIONAL_DECILE','F1')
    non_target_cnt=decile_data.groupBy('NATIONAL_DECILE').agg(func.sum("prob").alias('predicted_target_sum'))
    non_target_cnt=non_target_cnt.withColumnRenamed('NATIONAL_DECILE','F2')
    overall_cnt=decile_data.groupBy('NATIONAL_DECILE').count().alias('Total')
    overall_cnt=overall_cnt.join(target_cnt,target_cnt.F1==overall_cnt.NATIONAL_DECILE)
    overall_cnt=overall_cnt.join(non_target_cnt,non_target_cnt.F2==overall_cnt.NATIONAL_DECILE)
    #overall_cnt=overall_cnt.withColumn("Actual %",(func.col('target_sum ')/func.col('target_sum').sum())*100)



    pf = overall_cnt.toPandas() 
    return pf

In [154]:
# Use to find min / max probabilitiy in each decile
def percents2(data, model_score, target):
    wAllReg =  Window.orderBy(desc('prob'))
    decile_data=data.withColumn('NATIONAL_DECILE',ntile(10).over(wAllReg))
    decile_data=decile_data.withColumn('non_target',1-decile_data['target'])

    target_cnt=decile_data.groupBy('NATIONAL_DECILE').agg(func.max('prob').alias('MAX'))
    target_cnt=target_cnt.withColumnRenamed('NATIONAL_DECILE','F1')
    non_target_cnt=decile_data.groupBy('NATIONAL_DECILE').agg(func.min("prob").alias('MIN'))
    non_target_cnt=non_target_cnt.withColumnRenamed('NATIONAL_DECILE','F2')
    overall_cnt=decile_data.groupBy('NATIONAL_DECILE').count().alias('Total')
    overall_cnt=overall_cnt.join(target_cnt,target_cnt.F1==overall_cnt.NATIONAL_DECILE)
    overall_cnt=overall_cnt.join(non_target_cnt,non_target_cnt.F2==overall_cnt.NATIONAL_DECILE)
    #overall_cnt=overall_cnt.withColumn("Actual %",(func.col('target_sum ')/func.col('target_sum').sum())*100)



    pf = overall_cnt.toPandas() 
    return pf
   

In [156]:
# Output min and max probability for each decile
chart = percents2(results_sf, results_sf.select('prob'), results_sf.select('target'))
chart.head(10)

,NATIONAL_DECILE,count,F1,MAX,F2,MIN
0,1,66457,1,0.100755,1,0.013829
1,2,66457,2,0.013829,2,0.008948
2,3,66457,3,0.008948,3,0.006829
3,4,66457,4,0.006829,4,0.004843
4,5,66457,5,0.004843,5,0.003527
5,6,66457,6,0.003527,6,0.002621
6,7,66457,7,0.002621,7,0.002065
7,8,66457,8,0.002065,8,0.001658
8,9,66456,9,0.001658,9,0.001363
9,10,66456,10,0.001363,10,0.001140


In [157]:
maxes = chart['MAX'].tolist()
mins = chart['MIN'].tolist() 
for Max in maxes: 
    print(Max) 
print("\n")
for Min in mins: 
    print(Min)

0.10075473033
0.0138293097114
0.00894751708622
0.00682931189465
0.00484269280924
0.0035270645651
0.00262098529287
0.00206517113219
0.00165797236787
0.00136286692441


0.0138293097114
0.00894763074821
0.00682931189465
0.00484269280924
0.0035270645651
0.00262098529287
0.00206517113219
0.00165797236787
0.00136286692441
0.00114039818511


In [158]:
# Output predicted vs actual per decile.
chart = percents(results_sf, results_sf.select('prob'), results_sf.select('target'))
chart.head(10)

,NATIONAL_DECILE,count,F1,target_sum_total,F2,predicted_target_sum
0,1,66457,1,934,1,1353.324107
1,2,66457,2,701,2,729.334279
2,3,66457,3,495,3,521.014821
3,4,66457,4,429,4,386.175771
4,5,66457,5,267,5,275.225666
5,6,66457,6,202,6,201.857264
6,7,66457,7,160,7,154.489164
7,8,66457,8,112,8,124.707019
8,9,66456,9,90,9,100.041269
9,10,66456,10,58,10,81.113275


In [159]:
target_sums = chart['target_sum_total'].tolist()
predicted_target_sums = chart['predicted_target_sum'].tolist()

target_count = 0.0
for item in target_sums: 
    target_count += item 
running_sum = 0 
for target_sum in target_sums: 
    output = (target_sum / target_count) 
    print(output)
    running_sum = running_sum + output 
print(running_sum) 


for predicted_target_sum in predicted_target_sums: 
    output = (predicted_target_sum / target_count) 
    print(output)

0.270881670534
0.203306264501
0.143561484919
0.124419953596
0.0774361948956
0.0585846867749
0.046403712297
0.0324825986079
0.0261020881671
0.0168213457077
1.0
0.392495390797
0.211523862962
0.151106386586
0.111999933548
0.0798218289781
0.0585432900928
0.0448054421175
0.0361679289689
0.0290142890243
0.0235247315911


## Profile

In [24]:
# Split sorted dataframe based on decile scores.
top_30 = df[:209592 ]
mid_30 = df[209592:419184]
bottom_40 = df[419184:698641]

In [26]:
# Number of customers in 3 deciles and 4 deciles.
total_30 = 209592.0
total_40 = 279456.0

In [29]:
# Age
count = 0
for item in bottom_40['first_indv_age']:
    try:
        item = int(item)
    except(ValueError, TypeError): 
        continue
    if item > 45: 
        count += 1 
print( count / total_40 * 100)

65.6965676171


In [32]:
# Income 
count = 0
for item in bottom_40['income_code_raw']:
    if item in ('F', 'G', 'H', 'I', 'J', 'K', 'L'): 
        count += 1 
print( count / total_40 * 100)


63.8247165922


In [35]:
# Home owner 
count = 0
for item in bottom_40['home_own_rent']:
    if item in ('Own', 'ML Own'): 
        count += 1 
print( count / total_40 * 100)

81.0188365968


In [38]:
# Children
count = 0
for item in bottom_40['children_num_hh']:
    try:
        item = int(item)
    except(ValueError, TypeError): 
        continue
    if item > 0: 
        count += 1 
print( count / total_40 * 100)

35.7802301615


In [41]:
# College 
count = 0
for item in bottom_40['first_indv_edu']:
    if item == 'COLG' or item == 'GRAD': 
        count += 1 
print( count / total_40 * 100)

50.5253063094


In [47]:
# SFU/ MDU 
count = 0
for item in top_30['nsd_master_dwell_type_group_biller']:
    if item == 'MDU': 
        count += 1 
print( count / total_30 * 100)

35.3381808466
